In [1]:
!pip install pyarrow mrjob

In [2]:
%%file solution.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class MRLinesCount(MRJob):
    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer1,
            ),
            MRStep(reducer=self.reducer2)
        ]

    def mapper(self, _, line):
        splitted = line.split('\" \"')
        character = splitted[1]
        line = splitted[-1].strip()[:-1]

        yield (character, (len(line), line))

    def combiner(self, character, lines):
        yield (character, max(lines))

    def reducer1(self, character, lines):
        yield None, (max(lines), character)

    def reducer2(self, _, lines_character):
        for line, character in sorted(lines_character, reverse=True):
            if "dialogue" not in character:   
                yield character, line[1]
        
if __name__ == "__main__":
    MRLinesCount.run()

Overwriting solution.py


# Проверяем локально

In [3]:
!python3 solution.py ../SW_EpisodeIV.txt
!python3 solution.py ../SW_EpisodeV.txt
!python3 solution.py ../SW_EpisodeVI.txt
!python3 solution.py ../SW_EpisodeIV.txt ../SW_EpisodeV.txt ../SW_EpisodeVI.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/solution.root.20231128.180948.211409
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/solution.root.20231128.180948.211409/output
Streaming final output from /tmp/solution.root.20231128.180948.211409/output...
"LEIA"	"General Kenobi, years ago you served my father in the Clone Wars.  Now he begs you to help him in his struggle against the Empire.  I regret that I am unable to present my father's request to you in person, but my ship has fallen under attack and I'm afraid my mission to bring you to Alderaan has failed.  I have placed information vital to the survival of the Rebellion into the memory systems of this R2 unit.  My father will know how to retrieve it.  You must see this droid safely delivered to him on Alderaan.  This is our most desperate hour.  Help me, Obi-Wan Kenobi, you're my only hope."
"BIGGS"	"I feel for you, Luke, you're going 

# Проверяем на кластере

In [4]:
!python3 solution.py -r hadoop hdfs://namenode:8020/SW_EpisodeIV.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/solution.root.20231128.180950.210192
uploading working dir files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.180950.210192/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.180950.210192/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar4688119308261644164/] [] /tmp/streamjob613139060844216118.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1701178253713_0017
  Total 

In [5]:
!python3 solution.py -r hadoop hdfs://namenode:8020/SW_EpisodeV.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/solution.root.20231128.181045.779005
uploading working dir files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.181045.779005/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.181045.779005/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar5133878571364725276/] [] /tmp/streamjob6692908376036949186.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1701178253713_0019
  Total

In [ ]:
!python3 solution.py -r hadoop hdfs://namenode:8020/SW_EpisodeVI.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/solution.root.20231128.181141.787180
uploading working dir files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.181141.787180/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/solution.root.20231128.181141.787180/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar7015335378131015102/] [] /tmp/streamjob1912859030344492566.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1701178253713_0021
  Total

In [ ]:
!python3 solution.py -r hadoop hdfs://namenode:8020/SW_EpisodeIV.txt hdfs://namenode:8020/SW_EpisodeV.txt hdfs://namenode:8020/SW_EpisodeVI.txt